# Portfolio simulation
#### using Geometric Brownian Motion (GBM) model
P. Glasserman, Monte Carlo methods in financial engineering. Vol. 53 (2013), Springer Science & Business Media.

In [1]:
# Asset Path
import numpy as np

def GBMsimulator(seed, So, mu, sigma, Cov, T, N):
    """
    Parameters
    seed:   seed of simulation
    So:     initial stocks' price
    mu:     expected return
    sigma:  volatility
    Cov:    covariance matrix
    T:      time period
    N:      number of increments
    """

    np.random.seed(seed)
    dim = np.size(So)
    t = np.linspace(0., T, int(N))
    A = np.linalg.cholesky(Cov)
    S = np.zeros([dim, int(N)])
    S[:, 0] = So
    for i in range(1, int(N)):    
        drift = (mu - 0.5 * sigma**2) * (t[i] - t[i-1])
        Z = np.random.normal(0., 1., dim)
        diffusion = np.matmul(A, Z) * (np.sqrt(t[i] - t[i-1]))
        S[:, i] = S[:, i-1]*np.exp(drift + diffusion)
    return S, t

In [2]:
import pandas_datareader as pdr
from datetime import datetime

intc = pdr.get_data_yahoo(symbols='INTC', start=datetime(2018, 1, 1), end=datetime(2020, 1, 1))
amd = pdr.get_data_yahoo(symbols='AMD', start=datetime(2018, 1, 1), end=datetime(2020, 1, 1))

TypeError: string indices must be integers

In [ ]:
#Visualize the closing price history
import matplotlib.pyplot as plt

plt.figure(figsize = (16,8))
plt.title('Close Price History', fontsize = 18)
plt.plot(intc['Adj Close'])
plt.plot(amd['Adj Close'])
plt.legend(['INTC', 'AMD'], loc = 'upper left', fontsize = 18)
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Close Price USD ($)', fontsize = 18)
plt.show()

In [ ]:

# Two-dimensional Case

seed = 22                       

dim = 2; T = 1; N = int(2.**9)

S0 = np.array([100, 100])

# Parameter Estimation

# logarithmic returns

intc['log_returns'] = np.log(intc['Adj Close']).diff()
amd['log_returns'] = np.log(amd['Adj Close']).diff()

# expected returns

mean_intc = intc['log_returns'][1:].mean()
mean_amd = amd['log_returns'][1:].mean()
mu = [mean_intc, mean_amd]

# volatility
std_intc = intc['log_returns'][1:].std()
std_amd = amd['log_returns'][1:].std()
sigma = np.array([std_intc, std_amd])

# covariance matrix
Cov = np.cov(intc['log_returns'][1:], amd['log_returns'][1:] )

stocks, time = GBMsimulator(seed, S0, mu, sigma, Cov, T, N)

# Visualize Simulation
import matplotlib.pyplot as plt

plt.figure(figsize = (16,8))
plt.title('Two-dimensional Correlated Geometric Brownian Motion', fontsize = 18)
plt.plot(time, stocks[0,:], label='Stock 1')
plt.plot(time, stocks[1,:], label='Stock 2')

plt.xlabel('Time', fontsize = 18)
plt.legend(['Stock 1', 'Stock 2'], loc = 'upper left', fontsize = 18)
plt.show()